# Ambulance Redeployment Simulation Runner

Use this notebook to execute the SimPy-based simulator across every redeployment rule and compare key KPIs (response time, coverage, utilization, queue length).

## Data note

This folder does not include simulated incident JSON datasets (for example: `day.json`, `week.json`). Only the prompt scripts (`prompt.sh`) are kept under `resources/simulated_records/`.

To run simulations, pass incident-history files using `history_paths=...` in `run_rules(...)` / `SimulationRunner.run(...)`.


In [ ]:
from __future__ import annotations

import sys
from pathlib import Path


def _resolve_repo_root() -> Path:
    """Find the repo root (expects `util/` next to `resources/`)."""
    base = Path.cwd().resolve()
    for candidate in (base,) + tuple(base.parents):
        util_dir = candidate / "util"
        resources_dir = candidate / "resources"
        if util_dir.is_dir() and resources_dir.is_dir():
            return candidate
    raise RuntimeError("Unable to locate repo root (expected util/ and resources/).");


REPO_ROOT = _resolve_repo_root()
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

In [ ]:
from pathlib import Path
from typing import Iterable, Sequence

import pandas as pd

from util.simulator.rules import RuleCatalog
from util.simulator.simulator import SimulationRunner


catalog = RuleCatalog()
runner = SimulationRunner()

ALL_RULE_IDS = tuple(catalog.ids())
DEFAULT_TEMPLATE = "day"
SIM_DATA_ROOT = REPO_ROOT / "resources" / "simulated_records"

def run_rules(
    rule_ids: Iterable[str] | None = None,
    *,
    template: str = DEFAULT_TEMPLATE,
    seed: int = 0,
    history_paths: Iterable[str | Path] | None = None,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Run the simulator for every rule and return summary + unit-level metrics."""

    ids = tuple(rule_ids) if rule_ids else ALL_RULE_IDS
    rows = []
    unit_rows = []
    for idx, rule_id in enumerate(ids, start=1):
        result = runner.run(
            rule_id,
            template,
            seed=seed + idx,
            history_paths=history_paths,
        )
        row = {"rule": rule_id, **result.metrics}
        row["seed"] = seed + idx
        row["template"] = template
        rows.append(row)
        for unit_entry in result.unit_utilization:
            unit_rows.append({"rule": rule_id, **unit_entry})
    summary_df = (
        pd.DataFrame(rows).set_index("rule").sort_values("average_response_minutes")
    )
    unit_df = pd.DataFrame(unit_rows)
    return summary_df, unit_df

In [ ]:
RULE_BUNDLES: dict[str, Sequence[str]] = {
}


def _paths(*names: str) -> list[Path]:
    return [SIM_DATA_ROOT / name for name in names]


SCENARIO_LIBRARY: dict[str, dict] = {
    "baseline_day": {
        "template": "day",
        "bundle": "simple",
        "paths": _paths("baseline/day.json"),
        "notes": "1-day FCFS benchmark",
    },
    "baseline_week": {
        "template": "week",
        "bundle": "simple",
        "paths": _paths("baseline/week.json"),
        "notes": "7-day FCFS reference",
    },
    "baseline_month": {
        "template": "month",
        "bundle": "simple",
        "paths": _paths("baseline/month.json"),
        "notes": "30-day FCFS reference",
    },
    "intermediate_day": {
        "template": "day",
        "bundle": "peak",
        "paths": _paths("intermediate/day.json"),
        "notes": "Dense day with multiple urban peaks",
    },
    "intermediate_week": {
        "template": "week",
        "bundle": "peak",
        "paths": _paths("intermediate/week.json"),
        "notes": "Urban-heavy week scenario",
    },
    "intermediate_month": {
        "template": "month",
        "bundle": "peak",
        "paths": _paths("intermediate/month.json"),
        "notes": "30-day mixed peaks",
    },
    "stress_day": {
        "template": "day",
        "bundle": "perfect",
        "paths": _paths("stress/day.json"),
        "notes": "High-demand stress test",
    },
    "stress_week": {
        "template": "week",
        "bundle": "perfect",
        "paths": _paths("stress/week.json"),
        "notes": "Aggressive peak redeploy workload",
    },
    "stress_month": {
        "template": "month",
        "bundle": "perfect",
        "paths": _paths("stress/month.json"),
        "notes": "Extreme redeploy workload",
    },
    "lightweight_day": {
        "template": "day",
        "bundle": "perfect",
        "paths": _paths("lightweight/day.json"),
        "notes": "Lean day aiming for sub-15 min responses",
    },
    "lightweight_week": {
        "template": "week",
        "bundle": "perfect",
        "paths": _paths("lightweight/week.json"),
        "notes": "Lean week scenario",
    },
    "lightweight_month": {
        "template": "month",
        "bundle": "perfect",
        "paths": _paths("lightweight/month.json"),
        "notes": "Lean month scenario",
    },
}


def run_scenario(
    scenario_name: str,
    *,
    seed: int = 100,
    rule_ids: Sequence[str] | None = None,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    config = SCENARIO_LIBRARY[scenario_name]
    selected_rules = rule_ids or config.get("rule_ids")
    if selected_rules is None:
        bundle = config.get("bundle")
        selected_rules = RULE_BUNDLES.get(bundle, ALL_RULE_IDS)
    return run_rules(
        rule_ids=selected_rules,
        template=config.get("template", DEFAULT_TEMPLATE),
        seed=seed,
        history_paths=config.get("paths"),
    )


In [ ]:
summary_df, unit_df = run_scenario("baseline_day", seed=100)
summary_df


In [ ]:
ax = summary_df.sort_values("average_response_minutes").plot(
    kind="bar",
    y=["average_response_minutes", "coverage_ratio"],
    secondary_y="coverage_ratio",
    figsize=(12, 6),
    title="Response Time vs Coverage by Rule",
)
ax.set_ylabel("Average Response (minutes)")
ax.right_ax.set_ylabel("Coverage ratio")
ax.grid(axis="y", linestyle="--", alpha=0.4)
ax


In [ ]:
def plot_utilization_heatmap(unit_df: pd.DataFrame, top_n: int = 10):
    if unit_df.empty:
        print("No unit-level data available.")
        return
    pivot = (
        unit_df.groupby(["rule", "home_base_id"])  # aggregate per base
        ["busy_ratio"]
        .mean()
        .reset_index()
        .pivot(index="rule", columns="home_base_id", values="busy_ratio")
    )
    pivot = pivot.fillna(0).sort_index()
    ax = pivot.iloc[:, :top_n].plot(
        kind="bar",
        stacked=True,
        figsize=(14, 6),
        title="Unit Utilization Contribution per Rule",
    )
    ax.set_ylabel("Busy ratio (fraction of horizon)")
    ax.legend(title="Home Base", bbox_to_anchor=(1.05, 1), loc="upper left")
    ax.grid(axis="y", linestyle="--", alpha=0.3)
    return ax

# plot_utilization_heatmap(unit_df)



## Compare demand profiles
Use this section to run the same rule set against multiple incident-history files (e.g., `sim_day_peak1pm.json` vs `sim_week_peak1pm.json`) and visualize the metrics side by side.


In [ ]:
def compare_history_profiles(
    profiles: dict[str, dict],
    *,
    default_template: str = DEFAULT_TEMPLATE,
    seed: int = 500,
    rule_ids: Iterable[str] | None = None,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Run `run_rules` for each profile definition and concatenate metrics."""

    frames: list[pd.DataFrame] = []
    unit_frames: list[pd.DataFrame] = []
    for label, config in profiles.items():
        profile_template = config.get("template", default_template)
        profile_rules = config.get("rule_ids")
        bundle = config.get("bundle")
        selected_rules = profile_rules or (
            RULE_BUNDLES.get(bundle) if bundle else rule_ids
        )
        paths = [
            SIM_DATA_ROOT / Path(path)
            if not isinstance(path, Path)
            else path
            for path in config.get("paths", [])
        ]
        summary_df, unit_df = run_rules(
            rule_ids=selected_rules,
            template=profile_template,
            seed=seed,
            history_paths=paths,
        )
        summary_df = summary_df.reset_index()
        summary_df["profile"] = label
        unit_df = unit_df.copy()
        unit_df["profile"] = label
        frames.append(summary_df)
        unit_frames.append(unit_df)
        seed += config.get("seed_step", 17)
    combined = pd.concat(frames, ignore_index=True)
    combined_unit = pd.concat(unit_frames, ignore_index=True)
    return combined, combined_unit


history_profiles = {
    "Baseline day": {
        "paths": _paths("baseline/day.json"),
        "template": "day",
        "bundle": "simple",
    },
    "Intermediate week": {
        "paths": _paths("intermediate/week.json"),
        "template": "week",
        "bundle": "peak",
    },
    "Stress month": {
        "paths": _paths("stress/month.json"),
        "template": "month",
        "bundle": "perfect",
    },
}

comparison_df, comparison_unit_df = compare_history_profiles(
    history_profiles, seed=300
)
comparison_df.head()

In [ ]:
def plot_side_by_side(df: pd.DataFrame, metric: str, *, figsize=(12, 5)):
    pivot = df.pivot(index="rule", columns="profile", values=metric)
    ax = pivot.plot(kind="bar", figsize=figsize, title=f"{metric.replace('_', ' ').title()} by profile")
    ax.set_ylabel(metric.replace("_", " ").title())
    ax.grid(axis="y", linestyle="--", alpha=0.3)
    return ax

plot_side_by_side(comparison_df, "average_response_minutes")
plot_side_by_side(comparison_df, "coverage_ratio")



In [ ]:
plot_side_by_side(comparison_df, "utilization")



In [ ]:
plot_utilization_heatmap(comparison_unit_df)



## Tips
- Use `run_scenario(...)` presets to pick a rule bundle + history paths. Call `SCENARIO_LIBRARY.keys()` to list available presets.
- Pass a subset of rule IDs to `run_scenario(..., rule_ids=["Rule_A", "Rule_Y"])` to benchmark only a few policies.
- To compare histories, add more `_paths(...)` entries that point to your incident-history JSON files. (See `resources/simulated_records/**/prompt.sh` for the generation commands/parameters.)
- `unit_df` and `comparison_unit_df` can help spot bases or hotspots that spend a lot of time redeployed.
